In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

# hyper parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 1e-3
batch_size = 16
epochs = 150

# dataset and transform
trainset = MNIST('./data', train=True, transform=transforms.ToTensor(), download=True)
testset = MNIST('./data', train=False, transform=transforms.ToTensor())
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# build network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28*1, 256, bias=False)
        self.fc2 = nn.Linear(256, 256, bias=False)
        self.fc3 = nn.Linear(256, 10, bias=False)

    def forward(self, x):
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# optimizer
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

print(model)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} \t Loss: {:.6f}'.format(epoch, loss.item()))
     
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Net(
  (fc1): Linear(in_features=784, out_features=256, bias=False)
  (fc2): Linear(in_features=256, out_features=256, bias=False)
  (fc3): Linear(in_features=256, out_features=10, bias=False)
)


In [2]:
for epoch in range(1, epochs + 1):
    if epoch == 50:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate*0.1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 	 Loss: 2.091418

Test set: Average loss: 1.9827, Accuracy: 6502/10000 (65%)

Train Epoch: 2 	 Loss: 1.207563

Test set: Average loss: 0.9289, Accuracy: 8004/10000 (80%)

Train Epoch: 3 	 Loss: 0.744150

Test set: Average loss: 0.5767, Accuracy: 8508/10000 (85%)

Train Epoch: 4 	 Loss: 0.584864

Test set: Average loss: 0.4589, Accuracy: 8772/10000 (88%)

Train Epoch: 5 	 Loss: 0.541357

Test set: Average loss: 0.4026, Accuracy: 8900/10000 (89%)

Train Epoch: 6 	 Loss: 0.702199

Test set: Average loss: 0.3699, Accuracy: 8976/10000 (90%)

Train Epoch: 7 	 Loss: 0.384854

Test set: Average loss: 0.3493, Accuracy: 9007/10000 (90%)

Train Epoch: 8 	 Loss: 0.093965

Test set: Average loss: 0.3328, Accuracy: 9052/10000 (91%)

Train Epoch: 9 	 Loss: 0.826749

Test set: Average loss: 0.3200, Accuracy: 9092/10000 (91%)

Train Epoch: 10 	 Loss: 0.286936

Test set: Average loss: 0.3091, Accuracy: 9116/10000 (91%)

Train Epoch: 11 	 Loss: 0.437202

Test set: Average loss: 0.2981, Acc


Test set: Average loss: 0.1296, Accuracy: 9606/10000 (96%)

Train Epoch: 89 	 Loss: 0.184941

Test set: Average loss: 0.1294, Accuracy: 9611/10000 (96%)

Train Epoch: 90 	 Loss: 0.014650

Test set: Average loss: 0.1292, Accuracy: 9610/10000 (96%)

Train Epoch: 91 	 Loss: 0.107940

Test set: Average loss: 0.1290, Accuracy: 9609/10000 (96%)

Train Epoch: 92 	 Loss: 0.076173

Test set: Average loss: 0.1290, Accuracy: 9610/10000 (96%)

Train Epoch: 93 	 Loss: 0.364049

Test set: Average loss: 0.1288, Accuracy: 9608/10000 (96%)

Train Epoch: 94 	 Loss: 0.463815

Test set: Average loss: 0.1286, Accuracy: 9611/10000 (96%)

Train Epoch: 95 	 Loss: 0.012456

Test set: Average loss: 0.1285, Accuracy: 9609/10000 (96%)

Train Epoch: 96 	 Loss: 0.020383

Test set: Average loss: 0.1283, Accuracy: 9611/10000 (96%)

Train Epoch: 97 	 Loss: 0.496829

Test set: Average loss: 0.1280, Accuracy: 9614/10000 (96%)

Train Epoch: 98 	 Loss: 0.048137

Test set: Average loss: 0.1281, Accuracy: 9607/10000 (96%)


In [3]:
torch.save(model.state_dict(), "./model/mlp_model_1layer_256")